In [73]:
from absbox import API,mkDeal
localAPI = API("http://localhost:8081",check=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
⠋ Connecting engine server -> http://localhost:8081
✅Connected, local lib:0.20.2, server:0.23.4


In [131]:
deal_data = {
    "name":"Multiple Pools with Mixed Asset"
    ,"dates":{"cutoff":"2021-06-01"
              ,"closing":"2021-07-15"
              ,"firstPay":"2021-08-26"
              ,"payFreq":["DayOfMonth",20]
              ,"poolFreq":"MonthEnd"
              ,"stated":"2030-01-01"}
    ,"pool":{"PoolA":{'assets':[["Mortgage"
                        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
                          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                          ,{"currentBalance":2200
                          ,"currentRate":0.08
                          ,"remainTerm":30
                          ,"status":"current"}]]},
             "PoolB":{'assets':[["Loan"
                              ,{"originBalance": 80000
                                ,"originRate": ["floater",0.045,{"index":"SOFR3M"
                                                                ,"spread":0.01
                                                                ,"reset":"QuarterEnd"}]
                                ,"originTerm": 60
                                ,"freq": "Quarterly"
                                ,"type": "i_p"
                                ,"originDate": "2021-02-01"}
                              ,{"currentBalance": 65000
                                ,"currentRate": 0.06
                                ,"remainTerm": 60
                                ,"status": "Current"}]]}
            }
    ,"accounts":{"acc01":{"balance":0}
                ,"acc02":{"balance":0}}
    ,"bonds":{"A1":{"balance":1000
                 ,"rate":0.07
                 ,"originBalance":1000
                 ,"originRate":0.07
                 ,"startDate":"2020-01-03"
                 ,"rateType":{"Fixed":0.08}
                 ,"bondType":{"Sequential":None}}
             ,"B":{"balance":1000
                     ,"rate":0.0
                     ,"originBalance":1000
                     ,"originRate":0.07
                     ,"startDate":"2020-01-03"
                     ,"rateType":{"Fixed":0.00}
                     ,"bondType":{"Equity":None}}}
    ,"fees":{"trusteeFee":{"type":{"fixFee":30}}
            ,"serviceFee":{"type":{"annualPctFee":[("poolBalance","PoolB"),0.02]}}
            ,"serviceFee2":{"type":{"byTable":["MonthEnd"
                                               ,("const",1)
                                               ,[(0,5),(2,10),(10,15)]]
                                   }}}
    ,"collect":[[["PoolA"],"CollectedCash","acc01"]
               ,[["PoolB"],"CollectedCash","acc02"]]
    ,"waterfall":{"Amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["calcAndPayFee","acc01",['serviceFee']]
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payPrinResidual","acc01",["B"]]
     ]}
    ,"status":("PreClosing","Amortizing")
}

In [132]:
from absbox import mkDeal
deal = mkDeal(deal_data)  ## now a generic class created

In [133]:
r = localAPI.run(deal
                 ,poolAssump = ("ByName"
                                ,{"PoolA":
                                      (("Mortgage",{"CDR":0.02} ,None, None, None)
                                       ,None
                                       ,None)
                                ,"PoolB":
                                  (("Loan",{"CDR":0.01} ,None, None, None)
                                   ,None
                                   ,None)}
                                )
                 ,runAssump = [("interest",("LIBOR6M",0.04)
                                           ,("SOFR3M",0.04))
                               ,("inspect",("MonthEnd",("poolBalance","PoolB"))
                                          ,("MonthEnd",("poolBalance","PoolA"))
                                )
                               
                              ]
                 ,read=True)

❌ Failed to talk to server http://localhost:8081/runDeal
⠋
❌Failed to get response from run
{
  "tag": "SingleRunReq",
  "contents": [
    {
      "tag": "UDeal",
      "contents": {
        "dates": {
          "tag": "PreClosingDates",
          "contents": [
            "2021-06-01",
            "2021-07-15",
            null,
            "2030-01-01",
            [
              "2021-07-15",
              {
                "tag": "MonthEnd"
              }
            ],
            [
              "2021-08-26",
              {
                "tag": "DayOfMonth",
                "contents": 20
              }
            ]
          ]
        },
        "name": "Multiple Pools with Mixed Asset",
        "status": {
          "tag": "PreClosing",
          "contents": {
            "tag": "Amortizing"
          }
        },
        "pool": {
          "tag": "MultiPool",
          "contents": {
            "PoolName:PoolA": {
              "assets": [
                {
           

In [124]:
r['pool']['flow']['PoolB']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,
2021-06-01,64840.92,0.00,0.00,0,0.00,0,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-15,64732.11,0.00,541.13,0,108.81,0,108.81,0.05,0.00,0.0,0.0,108.81,0.0,108.81
2021-07-31,64732.11,0.00,0.00,0,0.00,0,0.00,0.00,0.00,0.0,0.0,108.81,0.0,108.81
2021-08-31,64676.88,0.00,274.65,0,55.23,0,55.23,0.05,0.00,0.0,0.0,164.04,0.0,164.04
2021-09-30,64621.70,0.00,274.42,0,55.18,0,55.18,0.05,0.00,0.0,0.0,219.22,0.0,219.22
2021-10-31,64568.35,0.00,265.34,0,53.35,0,53.35,0.05,0.00,0.0,0.0,272.57,0.0,272.57
2021-11-30,64513.26,0.00,273.96,0,55.09,0,55.09,0.05,0.00,0.0,0.0,327.66,0.0,327.66
2021-12-31,64460.00,0.00,264.90,0,53.26,0,53.26,0.05,0.00,0.0,0.0,380.92,0.0,380.92
2022-01-31,64405.01,0.00,273.50,0,54.99,0,54.99,0.05,0.00,0.0,0.0,435.91,0.0,435.91


In [119]:
r['pool']['flow']['PoolA']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-06-01,1989.40,0.00,0.00,0,0.00,0,0.00,0.00,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-15,1847.71,135.10,25.99,0,6.59,0,6.59,0.08,None,None,135.10,0.0,0.0,6.59,0.0,6.59
2021-07-31,1847.71,0.00,0.00,0,0.00,0,0.00,0.00,None,None,135.10,0.0,0.0,6.59,0.0,6.59
2021-08-31,1776.50,68.05,12.29,0,3.16,0,3.16,0.08,None,None,203.15,0.0,0.0,9.75,0.0,9.75
2021-09-30,1705.08,68.38,11.82,0,3.04,0,3.04,0.08,None,None,271.53,0.0,0.0,12.79,0.0,12.79
2021-10-31,1633.53,68.73,11.34,0,2.82,0,2.82,0.08,None,None,340.26,0.0,0.0,15.61,0.0,15.61
2021-11-30,1561.67,69.06,10.87,0,2.80,0,2.80,0.08,None,None,409.32,0.0,0.0,18.41,0.0,18.41
2021-12-31,1489.67,69.41,10.39,0,2.59,0,2.59,0.08,None,None,478.73,0.0,0.0,21.00,0.0,21.00
2022-01-31,1417.37,69.75,9.91,0,2.55,0,2.55,0.08,None,None,548.48,0.0,0.0,23.55,0.0,23.55


In [115]:
r['accounts']['acc02']

,balance,change,memo
date,,,
2021-07-15,541.13,541.13,<PoolPoolName:PoolB:CollectedCash>
2021-07-31,541.13,0.00,<PoolPoolName:PoolB:CollectedCash>
2021-08-31,815.78,274.65,<PoolPoolName:PoolB:CollectedCash>
2021-09-30,1090.20,274.42,<PoolPoolName:PoolB:CollectedCash>
2021-10-31,1355.54,265.34,<PoolPoolName:PoolB:CollectedCash>
2021-11-30,1629.50,273.96,<PoolPoolName:PoolB:CollectedCash>
2021-12-31,1894.40,264.90,<PoolPoolName:PoolB:CollectedCash>
2022-01-31,2167.90,273.50,<PoolPoolName:PoolB:CollectedCash>
2022-02-28,2441.16,273.26,<PoolPoolName:PoolB:CollectedCash>
